In [0]:
# stock_name
# 

# -> init table zaciagniecie danych od do ->
# -> increamental load -> zobaczyc jakie mamy najnowsze dane dla tej akcji np 2025-02-06   -> ticker od do i jaka akacja -> od 2025-02-06 do dzien dzisiejszy akcja 


In [0]:
#todo read input pramiters
# stock_name = 'NVDA'
# start_date = '2010-01-01'
# end_date = '2025-02-06'

dbutils.widgets.text("stock_name", 'def')
stock_name = dbutils.widgets.get("stock_name")
print(stock_name)
dbutils.widgets.dropdown("stocks", "NVDA", ["NVDA", "AAPL", "AMZN", "GOOG", "FB", "TSLA", "MSFT", "INTC", "CSCO", "QCOM"])

In [0]:
%sql
select max(date_time) from machine_learning.stock_forecaster.bronze_stock_data where stocks_name = 'NVDA'

In [0]:
from datetime import timedelta, datetime
df = spark.sql("select max(date_time) from machine_learning.stock_forecaster.bronze_stock_data where stocks_name = 'NVDA'")
display(df)
start_date = df.collect()[0][0]
print(type(start_date))
start_time_plus_day = (start_date + timedelta(days=1)).strftime("%Y-%m-%d")
print(start_time_plus_day)


In [0]:
import yfinance as yf
print(stock_name, start_date.strftime("%Y-%m-%d"), datetime.now().strftime("%Y-%m-%d"))

new_data = yf.download(stock_name, start=start_time_plus_day, end=datetime.now().strftime("%Y-%m-%d"))

print(new_data.reset_index().head())

In [0]:
new_data = new_data.reset_index()
new_columns =  ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
new_data.columns = new_columns
df = spark.createDataFrame(new_data)
# new columns that is date from Data (YYYY-MM-DD), ingestion_timestamp, ingestion_date, and stocks_name that is costant NVDA
import pyspark.sql.functions as f
df = df.withColumn('date_time', f.to_date(f.col('Date'))).withColumn('ingestion_timestamp', f.current_timestamp()).withColumn('ingestion_date', f.current_date()).withColumn('stocks_name', f.lit(stock_name))


In [0]:
df.write.format("delta").mode("append").saveAsTable("machine_learning.stock_forecaster.bronze_stock_data")

In [0]:
df.write.format("delta").mode("append").saveAsTable("machine_learning.stock_forecaster.bronze_stock_data") # adding again to create duplicates

In [0]:
df_read = spark.read.format("delta").table("machine_learning.stock_forecaster.bronze_stock_data")
display(df_read)

In [0]:
%sql
select * from machine_learning.stock_forecaster.bronze_stock_data version as of 2

In [0]:
%sql

RESTORE table machine_learning.stock_forecaster.bronze_stock_data version as of 2